In [ ]:
# ============================================================
# 1) Instalación de dependencias
# ============================================================
!pip -q install facenet-pytorch torch torchvision --upgrade

# ============================================================
# 2) Importaciones
# ============================================================
from google.colab import files
from PIL import Image
import torch
import numpy as np
import pandas as pd
import itertools
from facenet_pytorch import MTCNN, InceptionResnetV1

# ============================================================
# 3) Subir imágenes (EXACTAMENTE 3)
# ============================================================
print("Sube exactamente 3 imágenes (JPG/PNG) con un rostro cada una…")
uploaded = files.upload()

filenames = list(uploaded.keys())
if len(filenames) != 3:
    raise ValueError(f"Debes subir exactamente 3 imágenes. Subiste: {len(filenames)}")

# ============================================================
# 4) Preparar detector y red de embeddings
# ============================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=20, post_process=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def load_and_embed(path):
    """
    - Abre imagen
    - Detecta y alinea el rostro con MTCNN
    - Obtiene embedding de 512-D con InceptionResnetV1
    """
    img = Image.open(path).convert('RGB')
    face_tensor = mtcnn(img)
    if face_tensor is None:
        raise ValueError(f"No se detectó rostro en: {path}.")
    face_tensor = face_tensor.unsqueeze(0).to(device)
    with torch.no_grad():
        emb = resnet(face_tensor).cpu().numpy().flatten()
    return emb

# ============================================================
# 5) Obtener embeddings de las 3 imágenes
# ============================================================
embeddings = {}
for name in filenames:
    embeddings[name] = load_and_embed(name)

# ============================================================
# 6) Solución Analítica
# ============================================================
def proyeccion_analitica(a, b):
    """
    Calcula alpha* y el error mínimo de la proyección analítica.
    """
    dot_product = np.dot(a, b)
    norm_b_squared = np.dot(b, b)
    if norm_b_squared == 0:
        return 0, np.inf
    alpha_star = dot_product / norm_b_squared
    projection = alpha_star * b
    error = np.linalg.norm(a - projection)**2
    return alpha_star, error

# ============================================================
# 7) Solución Iterativa (Gradiente Descendente)
# ============================================================
def proyeccion_gradiente_descendente(a, b, learning_rate=0.01, num_iterations=1000):
    """
    Iterativamente encuentra alpha que minimiza ||a - alpha*b||^2
    """
    alpha = 0.0
    alpha_history = []
    for i in range(num_iterations):
        error_vector = a - alpha * b
        gradient = -2 * np.dot(error_vector, b)
        alpha = alpha - learning_rate * gradient
        alpha_history.append(alpha)
    projection = alpha * b
    error = np.linalg.norm(a - projection)**2
    return alpha, error, alpha_history

# ============================================================
# 8) Comparar todas las parejas
# ============================================================
pairs = list(itertools.combinations(filenames, 2))
rows = []
for (fa, fb) in pairs:
    ea, eb = embeddings[fa], embeddings[fb]

    # --- Analítica ---
    alpha_a, error_a = proyeccion_analitica(ea, eb)
    alpha_b, error_b = proyeccion_analitica(eb, ea)
    error_analitico = (error_a + error_b)/2

    # --- Iterativa ---
    alpha_a_gd, error_a_gd, _ = proyeccion_gradiente_descendente(ea, eb)
    alpha_b_gd, error_b_gd, _ = proyeccion_gradiente_descendente(eb, ea)
    error_iterativo = (error_a_gd + error_b_gd)/2

    rows.append({
        "Par": f"{fa} ↔ {fb}",
        "Error Analítico": error_analitico,
        "Error Iterativo": error_iterativo,
        "Alpha Analítico (a→b)": alpha_a,
        "Alpha Analítico (b→a)": alpha_b,
        "Alpha Final (GD a→b)": alpha_a_gd,
        "Alpha Final (GD b→a)": alpha_b_gd
    })

df = pd.DataFrame(rows).sort_values(by="Error Analítico")
print("\n=== Resultados de las Comparaciones ===")
display(df)

# ============================================================
# 9) Conclusión final
# ============================================================
best_pair = df.iloc[0]["Par"]
print(f"\nPAREJA MÁS PROBABLE (misma persona): {best_pair}")


Sube exactamente 3 imágenes (JPG/PNG) con un rostro cada una…


Saving Cristiano 2.jpg to Cristiano 2.jpg
Saving Cristiano.jpg to Cristiano (4).jpg
Saving messi 2.jpg to messi 2 (2).jpg

=== Resultados de las Comparaciones ===


,Par,Error Analítico,Error Iterativo,Alpha Analítico (a→b),Alpha Analítico (b→a),Alpha Final (GD a→b),Alpha Final (GD b→a)
0,Cristiano 2.jpg ↔ Cristiano (4).jpg,0.244112,0.244112,0.869419,0.869418,0.869418,0.869418
2,Cristiano (4).jpg ↔ messi 2 (2).jpg,0.891726,0.891726,0.329050,0.329050,0.329050,0.329050
1,Cristiano 2.jpg ↔ messi 2 (2).jpg,0.972576,0.972576,0.165602,0.165602,0.165602,0.165602



PAREJA MÁS PROBABLE (misma persona): Cristiano 2.jpg ↔ Cristiano (4).jpg


In [ ]:
# ============================================================
# 1) Instalación de dependencias
# ============================================================
# Se instalan las librerías necesarias: facenet-pytorch (para embeddings de rostros),
# torch (PyTorch), y torchvision (utilidades para trabajar con imágenes).
!pip -q install facenet-pytorch torch torchvision --upgrade

# ============================================================
# 2) Importaciones
# ============================================================
from google.colab import files        # Para subir imágenes en Colab
from PIL import Image                 # Para abrir y manipular imágenes
import torch                          # Librería principal para redes neuronales
import numpy as np                    # Para operaciones matemáticas y vectores
import pandas as pd                   # Para mostrar los resultados en tablas
import itertools                      # Para generar todas las combinaciones de pares
from facenet_pytorch import MTCNN, InceptionResnetV1  # Detector y modelo de embeddings

# ============================================================
# 3) Subir imágenes (EXACTAMENTE 3)
# ============================================================
print("Sube exactamente 3 imágenes (JPG/PNG) con un rostro cada una…")
uploaded = files.upload()             # Abre un selector de archivos en Colab

filenames = list(uploaded.keys())     # Guarda los nombres de los archivos subidos
if len(filenames) != 3:               # Validación: deben ser 3 imágenes
    raise ValueError(f"Debes subir exactamente 3 imágenes. Subiste: {len(filenames)}")

# ============================================================
# 4) Preparar detector y red de embeddings
# ============================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Usa GPU si está disponible, de lo contrario CPU.

mtcnn = MTCNN(image_size=160, margin=20, post_process=True, device=device)
# MTCNN = Multi-task Cascaded Convolutional Neural Networks
# Se encarga de detectar y alinear el rostro en la imagen.

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
# InceptionResnetV1 = red neuronal entrenada en el dataset VGGFace2.
# Convierte un rostro en un vector de 512 dimensiones (embedding).

def load_and_embed(path):
    """
    - Abre la imagen desde el disco
    - Detecta y alinea el rostro con MTCNN
    - Genera un embedding de 512 dimensiones con InceptionResnetV1
    """
    img = Image.open(path).convert('RGB')       # Carga la imagen y asegura formato RGB
    face_tensor = mtcnn(img)                    # Detecta el rostro
    if face_tensor is None:                     # Si no se detecta rostro, error
        raise ValueError(f"No se detectó rostro en: {path}.")
    face_tensor = face_tensor.unsqueeze(0).to(device)
    # Agrega dimensión batch y envía a GPU/CPU
    with torch.no_grad():                       # No calculamos gradientes (solo inferencia)
        emb = resnet(face_tensor).cpu().numpy().flatten()
        # Embedding como vector 1D de 512 valores
    return emb

# ============================================================
# 5) Obtener embeddings de las 3 imágenes
# ============================================================
embeddings = {}
for name in filenames:                          # Para cada archivo subido
    embeddings[name] = load_and_embed(name)     # Guardamos el embedding correspondiente

# ============================================================
# 6) Solución Analítica
# ============================================================
def proyeccion_analitica(a, b):
    """
    Calcula alpha* y el error mínimo de la proyección analítica.
    Proyección: proj_b(a) = ((a·b)/(b·b)) * b
    """
    dot_product = np.dot(a, b)                  # Producto punto a·b
    norm_b_squared = np.dot(b, b)               # Norma al cuadrado ||b||^2
    if norm_b_squared == 0:                     # Evitar división por cero
        return 0, np.inf
    alpha_star = dot_product / norm_b_squared   # Escalar óptimo (alpha*)
    projection = alpha_star * b                 # Vector proyectado
    error = np.linalg.norm(a - projection)**2   # Error cuadrático entre a y su proyección
    return alpha_star, error

# ============================================================
# 7) Solución Iterativa (Gradiente Descendente)
# ============================================================
def proyeccion_gradiente_descendente(a, b, learning_rate=0.01, num_iterations=1000):
    """
    Usa gradiente descendente para encontrar alpha que minimiza ||a - alpha*b||^2
    """
    alpha = 0.0                                 # Inicializamos alpha en 0
    alpha_history = []                          # Guardamos historial de alphas
    for i in range(num_iterations):             # Iteraciones de GD
        error_vector = a - alpha * b            # Vector de error
        gradient = -2 * np.dot(error_vector, b) # Gradiente = -2 * (a - alpha*b)·b
        alpha = alpha - learning_rate * gradient# Actualización de alpha
        alpha_history.append(alpha)             # Guardamos el valor de alpha
    projection = alpha * b                      # Proyección final
    error = np.linalg.norm(a - projection)**2   # Error final
    return alpha, error, alpha_history

# ============================================================
# 8) Comparar todas las parejas
# ============================================================
pairs = list(itertools.combinations(filenames, 2)) # Generamos las 3 posibles parejas
rows = []
for (fa, fb) in pairs:                           # Para cada par de imágenes
    ea, eb = embeddings[fa], embeddings[fb]      # Sus embeddings

    # --- Analítica ---
    alpha_a, error_a = proyeccion_analitica(ea, eb) # Proyección de a sobre b
    alpha_b, error_b = proyeccion_analitica(eb, ea) # Proyección de b sobre a
    error_analitico = (error_a + error_b)/2         # Promedio de errores

    # --- Iterativa ---
    alpha_a_gd, error_a_gd, _ = proyeccion_gradiente_descendente(ea, eb) # a→b
    alpha_b_gd, error_b_gd, _ = proyeccion_gradiente_descendente(eb, ea) # b→a
    error_iterativo = (error_a_gd + error_b_gd)/2   # Promedio de errores

    # Guardamos resultados en la tabla
    rows.append({
        "Par": f"{fa} ↔ {fb}",
        "Error Analítico": error_analitico,
        "Error Iterativo": error_iterativo,
        "Alpha Analítico (a→b)": alpha_a,
        "Alpha Analítico (b→a)": alpha_b,
        "Alpha Final (GD a→b)": alpha_a_gd,
        "Alpha Final (GD b→a)": alpha_b_gd
    })

# Creamos un DataFrame ordenado por menor error (analítico)
df = pd.DataFrame(rows).sort_values(by="Error Analítico")
print("\n=== Resultados de las Comparaciones ===")
display(df)   # Mostramos la tabla de resultados

# ============================================================
# 9) Conclusión final
# ============================================================
best_pair = df.iloc[0]["Par"]   # La pareja con menor error
print(f"\nPAREJA MÁS PROBABLE (misma persona): {best_pair}")


Sube exactamente 3 imágenes (JPG/PNG) con un rostro cada una…


Saving Cristiano 2.jpg to Cristiano 2 (2).jpg
Saving Cristiano.jpg to Cristiano (6).jpg
Saving messi 1.jpg to messi 1 (4).jpg

=== Resultados de las Comparaciones ===


,Par,Error Analítico,Error Iterativo,Alpha Analítico (a→b),Alpha Analítico (b→a),Alpha Final (GD a→b),Alpha Final (GD b→a)
0,Cristiano 2 (2).jpg ↔ Cristiano (6).jpg,0.244112,0.244112,0.869419,0.869418,0.869418,0.869418
2,Cristiano (6).jpg ↔ messi 1 (4).jpg,0.912173,0.912173,0.296356,0.296356,0.296356,0.296356
1,Cristiano 2 (2).jpg ↔ messi 1 (4).jpg,0.969726,0.969726,0.173994,0.173994,0.173994,0.173994



PAREJA MÁS PROBABLE (misma persona): Cristiano 2 (2).jpg ↔ Cristiano (6).jpg
